***최초 작성일 : 22.09.21***<br>

***최종 작성일 : 22.09.28***

# Stacking Test-Sklearn, XGBoost, CatBoost, LightGBM

##### "이유한님의 캐글 코리아 캐글 스터디 커널 커리큘럼"에 따라 필사한 내용입니다.

- 필사 노트북의 원 출처 : https://www.kaggle.com/code/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm/script

-------

## 1. Import package

In [1]:
# Stacking Strater based on Allstate Faron's Script
# - https://www.kaggle.com/mmueller/allstate-claims-severity/stacking-starter/run/390867
# Preprocessing from ogrellier
# - https://www.kaggle.com/ogrellier/good-fun-with-ligthgbm

import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
# from sklearn.cross_validation import KFold # cross_validation은 없어진 하위 모듈
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import gc

In [16]:
# constant
NFOLDS = 3
SEED = 0
NROWS = None

# data load
data = pd.read_csv("./dataset/home-credit-default-risk/application_train.csv")
test = pd.read_csv("./dataset/home-credit-default-risk/application_test.csv")
prev = pd.read_csv("./dataset/home-credit-default-risk/previous_application.csv")

In [17]:
data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
prev.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# object 타입 변수만 가져옴.
# label encoding을 하기 위함.
categorical_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

# label encoding
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    print(indexer)
    test[f_] = indexer.get_indexer(test[f_])
    
gc.enable()

# target 지정
y_train = data['TARGET']
del data['TARGET']

Index(['Cash loans', 'Revolving loans'], dtype='object')
Index(['M', 'F', 'XNA'], dtype='object')
Index(['N', 'Y'], dtype='object')
Index(['Y', 'N'], dtype='object')
Index(['Unaccompanied', 'Family', 'Spouse, partner', 'Children', 'Other_A',
       'Other_B', 'Group of people'],
      dtype='object')
Index(['Working', 'State servant', 'Commercial associate', 'Pensioner',
       'Unemployed', 'Student', 'Businessman', 'Maternity leave'],
      dtype='object')
Index(['Secondary / secondary special', 'Higher education',
       'Incomplete higher', 'Lower secondary', 'Academic degree'],
      dtype='object')
Index(['Single / not married', 'Married', 'Civil marriage', 'Widow',
       'Separated', 'Unknown'],
      dtype='object')
Index(['House / apartment', 'Rented apartment', 'With parents',
       'Municipal apartment', 'Office apartment', 'Co-op apartment'],
      dtype='object')
Index(['Laborers', 'Core staff', 'Accountants', 'Managers', 'Drivers',
       'Sales staff', 'Cleaning staff'

In [25]:
data.tail(100)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307411,456144,0,1,0,0,0,65250.0,123637.5,8734.5,112500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
307412,456145,0,1,0,1,0,162000.0,900000.0,29034.0,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
307413,456146,0,1,0,0,0,99000.0,269550.0,14242.5,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
307414,456147,0,1,1,0,2,90000.0,1078200.0,31653.0,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
307415,456148,0,0,1,0,0,180000.0,589045.5,30204.0,508500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,1,0,157500.0,254700.0,27558.0,225000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,1,0,0,0,72000.0,269550.0,12001.5,225000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,1,0,0,0,153000.0,677664.0,29979.0,585000.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,0,1,0,0,0,171000.0,370107.0,20205.0,319500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,0,1,0,0,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,0,0,0,0,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,0,0,1,0,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,0,1,0,0,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,0,0,1,1,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
###################################################################
# Plese don't do this at home  
# Averaging factorized categorical features defeats my own resoning
###################################################################
# prev 데이터에서 마찬가지로 object 변수만을 가져옴
prev_cat_features = [
    f_ for f_ in prev.columns if prev[f_].dtype == 'object'
]

# label encoding
for f_ in prev_cat_features:
    prev[f_], _ = pd.factorize(prev[f_])

# 
avg_prev = prev.groupby('SK_ID_CURR').mean()
cnt_prev = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()

# 'nb_app' column 추가(?)
avg_prev['nb_app'] = cnt_prev['SK_ID_PREV']
del avg_prev['SK_ID_PREV']

In [5]:
x_train = data.merge(right = avg_prev.reset_index(), how = 'left', on = 'SK_ID_CURR')
x_test = test.merge(right = avg_prev.reset_index(), how = 'left', on = 'SK_ID_CURR')

x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

ntrain = x_train.shape[0]
ntest = x_test.shape[0]

exclouded_feats = ['SK_ID_CURR']
features = [f_ for f_ in x_train.columns if f_ not in exclouded_feats]

x_train = x_train[features]
x_test = x_test[features]

kf = KFold(n_splits = NFOLDS, shuffle = True, random_state = SEED)

In [6]:
class SklearnWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_state'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]
    
class CatboostWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['random_seed'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]
    
class LightGBMWrapper(object):
    def __init__(self, clf, seed = 0, params = None):
        params['feature_fraction_seed'] = seed
        params['bagging_seed'] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
        
    def predict(self, x):
        return self.clf.predict_proba(x)[:, 1]
    
class XgbWrapper(object):
    def __init__(self, seed = 0, params = None):
        self.param = params
        self.param['seed'] = seed
        self.nrounds = params.pop('nround', 250)
        
    def train(self, x_train, y_train):
        dtrain = xgb.DMatrix(x_train, label = y_train)
        self.gbdt = xgb.train(self.param, dtrain, self.nrounds)
        
    def predict(self, x):
        return self.gbdt.predict(xgb.DMatrix(x))
    
# k-fold를 통해 stacking할 데이터 셋을 만듬.
def get_oof(clf):
    oof_train = np.zeros((ntrain, ))
    oof_test = np.zeros((ntest, ))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]
        
        clf.train(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        
    oof_test[:] = oof_test_skf.mean(axis = 0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [7]:
# 트리 알고리즘들의 파라미터를 dictionary형으로 미리 지정
et_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

rf_params = {
    'n_jobs' : 16,
    'n_estimators' : 200,
    'max_features' : 0.5,
    'max_depth' : 12,
    'min_samples_leaf' : 2
}

xgb_params = {
    'seed' : 0,
    'colsample_bytree' : 0.7,
    'silent' : 1,
    'subsample' : 0.7,
    'learning_rate' : 0.075,
    'objective' : 'binary:logistic',
    'max_depth' : 4,
    'num_parallel_tree' : 1,
    'min_child_weight' : 1,
    'nround' : 200
}

catboost_params = {
    'iterations' : 200,
    'learning_rate' : 0.5,
    'depth' : 3,
    'l2_leaf_reg' : 40,
    'bootstrap_type' : 'Bernoulli',
    'subsample' : 0.7,
    'scale_pos_weight' : 5,
    'eval_metric' : 'AUC',
    'od_type' : 'Iter',
    'allow_writing_files' : False
}

lightgbm_params = {
    'n_estimators' : 200,
    'learning_rate' : 0.1,
    'num_leaves' : 123,
    'colsample_bytree' : 0.8,
    'subsample' : 0.9,
    'max_depth' : 15,
    'reg_alpha' : 0.1,
    'reg_lambda' : 0.1,
    'min_split_gain' : 0.01,
    'min_child_weght' : 2
}

In [8]:
# 객체 선언
xg = XgbWrapper(seed = SEED, params = xgb_params)
et = SklearnWrapper(clf = ExtraTreesClassifier, seed = SEED, params = et_params)
rf = SklearnWrapper(clf = RandomForestClassifier, seed = SEED, params = rf_params)
cb = CatboostWrapper(clf = CatBoostClassifier, seed = SEED, params = catboost_params)
lg = LightGBMWrapper(clf = LGBMClassifier, seed = SEED, params = lightgbm_params)

# stacking을 위해 학습 후 나온 예측 결과를 다시 데이터 셋 형태로 만들어줌
# 3 fold로 학습 후 각 fold 별 예측 결과를 저장
xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

[10:28:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:29:18] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:29:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm

0:	total: 17.6ms	remaining: 3.5s
1:	total: 32.9ms	remaining: 3.25s
2:	total: 48.2ms	remaining: 3.16s
3:	total: 62.9ms	remaining: 3.08s
4:	total: 79.9ms	remaining: 3.11s
5:	total: 94.5ms	remaining: 3.06s
6:	total: 110ms	remaining: 3.03s
7:	total: 124ms	remaining: 2.99s
8:	total: 138ms	remaining: 2.94s
9:	total: 153ms	remaining: 2.91s
10:	total: 171ms	remaining: 2.93s
11:	total: 186ms	remaining: 2.91s
12:	total: 201ms	remaining: 2.89s
13:	total: 216ms	remaining: 2.86s
14:	total: 232ms	remaining: 2.86s
15:	total: 249ms	remaining: 2.87s
16:	total: 265ms	remaining: 2.86s
17:	total: 280ms	remaining: 2.83s
18:	total: 293ms	remaining: 2.79s
19:	total: 308ms	remaining: 2.77s
20:	total: 322ms	remaining: 2.74s
21:	total: 337ms	remaining: 2.73s
22:	total: 353ms	remaining: 2.72s
23:	total: 368ms	remaining: 2.7s
24:	total: 382ms	remaining: 2.67s
25:	total: 397ms	remaining: 2.65s
26:	total: 410ms	remaining: 2.63s
27:	total: 428ms	remaining: 2.63s
28:	total: 460ms	remaining: 2.71s
29:	total: 478ms	rem

40:	total: 642ms	remaining: 2.49s
41:	total: 658ms	remaining: 2.48s
42:	total: 676ms	remaining: 2.47s
43:	total: 690ms	remaining: 2.45s
44:	total: 705ms	remaining: 2.43s
45:	total: 720ms	remaining: 2.41s
46:	total: 735ms	remaining: 2.39s
47:	total: 751ms	remaining: 2.38s
48:	total: 768ms	remaining: 2.37s
49:	total: 783ms	remaining: 2.35s
50:	total: 800ms	remaining: 2.34s
51:	total: 817ms	remaining: 2.33s
52:	total: 831ms	remaining: 2.3s
53:	total: 846ms	remaining: 2.29s
54:	total: 864ms	remaining: 2.28s
55:	total: 879ms	remaining: 2.26s
56:	total: 894ms	remaining: 2.24s
57:	total: 908ms	remaining: 2.22s
58:	total: 922ms	remaining: 2.2s
59:	total: 939ms	remaining: 2.19s
60:	total: 954ms	remaining: 2.17s
61:	total: 972ms	remaining: 2.16s
62:	total: 987ms	remaining: 2.15s
63:	total: 1s	remaining: 2.13s
64:	total: 1.01s	remaining: 2.11s
65:	total: 1.03s	remaining: 2.09s
66:	total: 1.04s	remaining: 2.07s
67:	total: 1.06s	remaining: 2.06s
68:	total: 1.08s	remaining: 2.04s
69:	total: 1.09s	re

In [9]:
# 각 알고리즘 별 RMSE
print("XG-CV : {}".format(sqrt(mean_squared_error(y_train, xg_oof_train))))
print("ET-CV : {}".format(sqrt(mean_squared_error(y_train, et_oof_train))))
print("RF-CV : {}".format(sqrt(mean_squared_error(y_train, rf_oof_train))))
print("CB-CV : {}".format(sqrt(mean_squared_error(y_train, cb_oof_train))))

# 모델 별 예측 결과를 하나로 합침
x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis = 1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis = 1)

print("{}, {}".format(x_train.shape, x_test.shape))

XG-CV : 0.25970741144869475
ET-CV : 0.26296502347137407
RF-CV : 0.2627260187688126
CB-CV : 0.3306000196333401
(307511, 4), (48744, 4)


In [10]:
# 하나로 합친 데이터 셋을 LR 알고리즘을 통해 다시 예측
logistic_regression = LogisticRegression()
logistic_regression.fit(x_train, y_train)

test["TARGET"] = logistic_regression.predict_proba(x_test)[:, 1]

# test[['SK_ID_CURR', 'TARGET']].to_csv('first_submission.csv', index = False, float_format = '%.8f')

In [11]:
test["TARGET"]

0        0.038541
1        0.095478
2        0.027545
3        0.047395
4        0.135914
           ...   
48739    0.041850
48740    0.079470
48741    0.027993
48742    0.050415
48743    0.208930
Name: TARGET, Length: 48744, dtype: float64

--------

### 1. 데이터셋 설명 

이 노트의 데이터는 변수에 대한 정보를 설명하지 않는다.

### 2. 분석 기법 등 이번 노트에서 쓰인 방법론

#### CatBoost

Catboost는 categorical boosting의 약자로, 주로 **범주형 데이터 처리**에 집중하고 **타겟 누수(target leakage)**를 개선한 알고리즘이다.<br>

타겟 누수는 예측 시점에서 사용할 수 없는 데이터가 데이터 셋에 포함되는 오류를 말한다. 즉 모델 학습 시에 출력 변수의 정보가 입력 변수에 존재하는 경우를 말한다.<br>
이러한 타겟 누수로 인해 prediction shift 문제가 발생할 수 있는데, **prediction shift는 학습 데이터의 조건부 확률(확률 분포)과 테스트 데이터의 조건부 확률(확률분포)이 다른 경우**를 말한다. 즉, 학습 데이터를 통한 예측과 테스트 데이터를 통한 예측이 전혀 다른 결과를 예측한다는 이야기이다.

따라서 CatBoost 알고리즘의 contribution은 아래와 같이 요약할 수 있다.
1. target leakage 문제 개선 → Ordered TS
2. predction shift 문제 개선 → Ordered boosting

일반적으로 범주형 변수를 모델에 학습하기 위해서는 숫자형 데이터로 변환하여야 하는데, 보통 one-hot encoding, label encoding 등을 사용한다. <br>  이때 주의할 점이 있는데, 만약 한 변수가 범주의 개수가 많다면 one-hot encoding의 경우 입력 변수의 개수가 기하급수로 늘어나게 된다.

이를 방지하기 위해 범주를 군집화하거나 다른 숫자형 데이터로 변환하는 과정들이 존재하는데 CatBoost의 논문에서는 **범주형 데이터를 숫자형 데이터로 변환하는 새로운 방법인 Target Statistics(TS) 방법을 제안**한다.

Target Statistics<br>
예시
<img src="image/TS.png" style="width: 300px;"/><br>
        출처 - https://velog.io/@lighthouse97/Cat-Boost%EC%9D%98-%EC%9D%B4%ED%95%B4<br>
        위 그림의 빈도가 낮은 범주인 E의 경우와 같이 노이즈가 발생할 수 있기 때문에 smoothing을 적용한다. 그 방법이 바로 greedy TS이다.
        
1. greedy TS<br>
    예시
    <img src="image/greedy_TS.png" style="width: 500px;"/><br>
    출처 - https://velog.io/@lighthouse97/Cat-Boost%EC%9D%98-%EC%9D%B4%ED%95%B4<br>
    그러나 위의 그림도 마찬가지로 타겟 누수가 발생한다. (train-test의 입력 데이터 x_i 분포가 다른 경우를 conditional shift라고 한다.)<br>
    CatBoost를 제안한 논문에서는 이 과정에서 첫번째 property를 제안한다.<br>
    
    **Property 1. 학습 데이터와 테스트 데이터셋에서 산출된 TS의 평균값이 같아야 한다.**
       
       
2. Holdout TS<br>
    Holdout TS는 conditional shift를 방지하기 위해 아예 학습 데이터셋을 두 개로 나누어 하나는 TS 계산, 하나는 학습에 활용하였는데 이렇게 한다면<br>
    Property 1.은 만족하지만 학습과 TS 계산에 데이터가 분리되어 데이터 양이 줄어든다는 단점이 있다. 이 과정에서 두번째 property를 제안한다.
    
    **Property 2. TS를 계산 및 모델 학습을 위해 모든 데이터를 효과적으로 사용해야 한다.**
    

3. Leave-one-out TS<br>
    해당 관측치를 제외하고 TS를 산출한다. 이 경우 Property 1.을 방지할 수는 있지만 데이터가 많은 경우 시간이 오래 걸린다.<br>
    
    
4. **Ordered TS**<br>
    예시
    <img src="image/Ordered_TS.png" style="width: 300px;"/><br>
    데이터셋에 임의의 시간 값(random permutation)을 부여하고, **k번 째 데이터의 TS를 계산할 때, k-1번 째 데이터까지 target value만를 사용**한다.<br>
    이를 통해 타겟 누수를 방지할 수 있다. 또한 모든 데이터셋을 사용할 수 있기 때문에 Property 1, 2를 모두 만족한다.<br>
    
따라서, CatBoost는 Ordered TS 방법을 사용한다.


**Ordered Boosting**<br>
기존의 그래디언트 부스팅 모델이 가지고 있는 고질적 문제인 타겟 누수로 인한 prediction shift(로 인한 오버피팅)을 다루기 위해 논문에서는 **Ordered boosting** 방법을 제안한다.

그래디언스 부스팅은 매 트리 생성시 잔차(residual)을 학습한다. 



참고한 사이트
- https://kicarussays.tistory.com/m/40
- https://velog.io/@lighthouse97/Cat-Boost%EC%9D%98-%EC%9D%B4%ED%95%B4

-----

#### 1. 공부한 내용
 → catboost 알고리즘을 처음 알아서 이에 대해 공부했다.

#### 2. 어려웠던 부분
 → 없음
 
#### 3. 느낀점 및 좋던 부분
 → 해당 노트는 전처리에 대한 영역이 거의 존재하지 않고 오직 stacking으로만 분석을 하다보니 아쉽다.
 
#### 4. 라이브러리 사용법
 → 
 
#### 5. 해당 커널의 특징(다른 커널과의 차별점)
 → 성능을 높이기 위해 stacking만을 사용하였고 그 밖에 다른 방법은 딱히 사용하지 않았다.
 
#### 6. 추후 공부 및 정리할 내용
 → catboost에 대한 내용